## 기본 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Thu Dec  8 08:26:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !unzip "/content/drive/MyDrive/copyer.zip" -d "/content/drive/MyDrive/augdata"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_184_0_1_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_184_0_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_184_1.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_184_1_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_184_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_0.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_0_1.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_0_1_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_0_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_1.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoyed_185_1_2.jpg  
  inflating: /content/drive/MyDrive/augdata/train2/30/samoye

In [3]:
import os
os.chdir("/content/drive/MyDrive")

## 모델 기본 세팅

In [4]:
from deeplab2.model.pixel_encoder import moat
import tensorflow as tf
import numpy as np

In [5]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

print(tf.test.gpu_device_name())

/device:GPU:0


In [6]:
# releases the global state: avoid clutter from old models and layers
K.clear_session()

In [7]:
# 초기 변수 설정
n_classes = 37

### 이미지 크기 설정
img_width, img_height = 256, 256

train_data_dir = '/content/drive/MyDrive/augdata/train'
train1_data_dir = '/content/drive/MyDrive/augdata/train1'
train2_data_dir = '/content/drive/MyDrive/augdata/train2'
validation_data_dir = '/content/drive/MyDrive/augdata/validation'
validation_data2_dir = '/content/drive/MyDrive/augdata/validation'
test_data_dir = '/content/drive/MyDrive/augdata/test'

nb_train_samples =  26496
nb_train1_samples =  13248
nb_train2_samples =  13248  
nb_validation_samples = 368
nb_validation2_samples = 368
nb_test_samples = 3668
batch_size = 32

In [8]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)
val_datagen2 = ImageDataGenerator(
    rescale=1./255,)
test_datagen = ImageDataGenerator(
    rescale=1./255,)

In [9]:
# flow_from_directory : Numpy Array Iterator 객체 생성
# 인자로 설정해주는 directory의 바로 하위 디렉토리 이름을 레이블이라고 간주, 그 디렉토리 아래의 파일들을 해당 레이블의 이미지들이라고 알아서 추측

import tensorflow as tf
import numpy as np
import cv2

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

train1_generator = train1_datagen.flow_from_directory(
    train1_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train2_generator = train2_datagen.flow_from_directory(
    train2_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    
  
validation2_generator = val_datagen.flow_from_directory(
    validation_data2_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)


Found 26496 images belonging to 37 classes.
Found 13248 images belonging to 37 classes.
Found 13248 images belonging to 37 classes.
Found 368 images belonging to 37 classes.
Found 368 images belonging to 37 classes.
Found 3668 images belonging to 37 classes.


In [10]:
from tensorflow.python.ops.clip_ops import clip_by_global_norm
!pip install adabelief_tf
from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras
from adabelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

## 모델 feat. 다연

In [11]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")
o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
out1 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer')(r1)

def create_model():
  return tf.keras.models.Model(moat1.input, out1)


In [17]:
with tf.device('/device:GPU:0'):
  model1=create_model()
  model1.load_weights('pet_final_disjoint1_3.hdf5')
  # moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")
  opt2 = AdaBeliefOptimizer(learning_rate=1e-6, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)
  model1.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='pet_final_disjoint1_4.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('pet_final_disjoint1_4.log')
  history = model1.fit_generator(train1_generator,
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=80,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-17-af9e71939828>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(train1_generator,


Epoch 1/80
414/414 [==============================] - ETA: 0s - loss: 0.0461 - sparse_categorical_accuracy: 0.9872
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.89205, saving model to pet_final_disjoint1_3.hdf5
414/414 [==============================] - 433s 907ms/step - loss: 0.0461 - sparse_categorical_accuracy: 0.9872 - val_loss: 0.5907 - val_sparse_categorical_accuracy: 0.8920
Epoch 2/80
414/414 [==============================] - ETA: 0s - loss: 0.0391 - sparse_categorical_accuracy: 0.9887
Epoch 2: val_sparse_categorical_accuracy improved from 0.89205 to 0.89773, saving model to pet_final_disjoint1_3.hdf5
414/414 [==============================] - 375s 904ms/step - loss: 0.0391 - sparse_categorical_accuracy: 0.9887 - val_loss: 0.5864 - val_sparse_categorical_accuracy: 0.8977
Epoch 3/80
414/414 [==============================] - ETA: 0s - loss: 0.0426 - sparse_categorical_accuracy: 0.9863
Epoch 3: val_sparse_categorical_accuracy improved from 0.89773 to 0.90341, s

KeyboardInterrupt: ignored

In [13]:
  moat1.save_weights('pet_moat1_final_disjoint1_4.h5')

In [ ]:
#### 'pet_moat1_final_disjoint1_4.hdf5' 가 91% 정도로 베스트 스코어.

In [18]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
out2 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer')(r2)

def create_model2():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  model2 = create_model2()
  model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='pet_final_disjoint2_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('pet_final_disjoint2_2.log')
  history = model2.fit_generator(train2_generator,
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=80,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


<ipython-input-18-d7cb6deb214c>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model2.fit_generator(train2_generator,


Epoch 1/80
414/414 [==============================] - ETA: 0s - loss: 1.7369 - sparse_categorical_accuracy: 0.7172
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.01989, saving model to pet_final_disjoint2_2.hdf5


ValueError: ignored

In [ ]:
  moat2.save_weights('pet_moat2_final_disjoint2_2.h5')

## 앙상블 V1

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('pet_moat1_final_disjoint1.h5')
moat4.load_weights('pet_moat2_final_disjoint2.h5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=512, activation='gelu')(z)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

opt2 = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_1.hdf5', verbose=1, save_best_only=True)
  csv_logger = CSVLogger('ensemble_1.hdf5.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
66/66 [==============================] - ETA: 0s - loss: 1.7782 - sparse_categorical_accuracy: 0.7814
Epoch 1: val_loss improved from inf to 1.38070, saving model to ensemble_1.hdf5
66/66 [==============================] - 589s 9s/step - loss: 1.7782 - sparse_categorical_accuracy: 0.7814 - val_loss: 1.3807 - val_sparse_categorical_accuracy: 0.8143
Epoch 2/30
66/66 [==============================] - ETA: 0s - loss: 0.4727 - sparse_categorical_accuracy: 0.9224
Epoch 2: val_loss did not improve from 1.38070
66/66 [==============================] - 62s 928ms/step - loss: 0.4727 - sparse_categorical_accuracy: 0.9224 - val_loss: 1.5756 - val_sparse_categorical_accuracy: 0.8143
Epoch 3/30
66/66 [==============================] - ETA: 0s - loss: 0.3024 - sparse_categorical_accuracy: 0.9396
Epoch 3: val_loss improved from 1.38070 to 1.21967, saving model to ensemble_1.hdf5
66/66 [==============================] - 67s 1s/step - loss: 0.3024 - sparse_categorical_accuracy: 0.9396 - val_

KeyboardInterrupt: ignored

### 앙상블 V1 fine tuning

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=512, activation='gelu')(z)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

en_model = create_model3()

moat3.trainable=False
moat4.trainable=False

en_model.load_weights('ensemble_1.hdf5')

moat3.trainable=True
moat4.trainable=True

opt3 = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10, restore_best_weights=True)])





Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-16-1ed9894f5f8a>:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/5
207/207 [==============================] - ETA: 0s - loss: 0.2262 - sparse_categorical_accuracy: 0.9484
Epoch 1: val_loss improved from inf to 0.85923, saving model to ensemble_1_fine.hdf5


ValueError: ignored

## 앙상블 V2

In [ ]:
moat3 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('pet_moat1_final_disjoint1_2.hdf5')
moat4.load_weights('pet_moat2_final_disjoint2_2.hdf5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=256, activation='gelu')(z)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('ensemble_2.hdf5.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer, keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10)])
  



<ipython-input-21-e1d4db9af2e7>:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/200
207/207 [==============================] - ETA: 0s - loss: 1.4750 - sparse_categorical_accuracy: 0.7965
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.83152, saving model to ensemble_2.hdf5
207/207 [==============================] - 92s 358ms/step - loss: 1.4750 - sparse_categorical_accuracy: 0.7965 - val_loss: 1.0279 - val_sparse_categorical_accuracy: 0.8315
Epoch 2/200
207/207 [==============================] - ETA: 0s - loss: 0.3687 - sparse_categorical_accuracy: 0.9266
Epoch 2: val_sparse_categorical_accuracy improved from 0.83152 to 0.86685, saving model to ensemble_2.hdf5
207/207 [==============================] - 73s 350ms/step - loss: 0.3687 - sparse_categorical_accuracy: 0.9266 - val_loss: 0.8111 - val_sparse_categorical_accuracy: 0.8668
Epoch 3/200
207/207 [==============================] - ETA: 0s - loss: 0.2863 - sparse_categorical_accuracy: 0.9423
Epoch 3: val_sparse_categorical_accuracy improved from 0.86685 to 0.87228, saving model to ensemb

### 앙상블 V2 FINE TUNE

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=256, activation='gelu')(z)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

en_model = create_model3()

moat3.trainable=False
moat4.trainable=False

en_model.load_weights('ensemble_2.hdf5')

moat3.trainable=True
moat4.trainable=True

opt3 = AdaBeliefOptimizer(learning_rate=1e-6, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10, restore_best_weights=True)])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-23-826b0ac297ab>:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/30
207/207 [==============================] - 316s 849ms/step - loss: 0.1679 - sparse_categorical_accuracy: 0.9662 - val_loss: 0.9419 - val_sparse_categorical_accuracy: 0.8967
Epoch 2/30
207/207 [==============================] - 172s 829ms/step - loss: 0.1316 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.9066 - val_sparse_categorical_accuracy: 0.8940
Epoch 3/30
207/207 [==============================] - 171s 827ms/step - loss: 0.1598 - sparse_categorical_accuracy: 0.9632 - val_loss: 0.8988 - val_sparse_categorical_accuracy: 0.8940
Epoch 4/30
207/207 [==============================] - 171s 827ms/step - loss: 0.1319 - sparse_categorical_accuracy: 0.9686 - val_loss: 0.8855 - val_sparse_categorical_accuracy: 0.8967
Epoch 5/30
207/207 [==============================] - 171s 826ms/step - loss: 0.1433 - sparse_categorical_accuracy: 0.9713 - val_loss: 0.9044 - val_sparse_categorical_accuracy: 0.9022
Epoch 6/30
207/207 [==============================] - 171s 827ms/step - loss: 0.

## 앙상블 V3 (쌩 softmax)

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('pet_moat1_final_disjoint1.h5')
moat4.load_weights('pet_moat2_final_disjoint2.h5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)
en_out = Dense(units=37, activation='softmax')(z)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_3.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('ensemble_3.hdf5.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer, keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10)])

<ipython-input-24-6413db5545f0>:39: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/200
207/207 [==============================] - ETA: 0s - loss: 1.5395 - sparse_categorical_accuracy: 0.7826
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.83424, saving model to ensemble_3.hdf5
207/207 [==============================] - 96s 346ms/step - loss: 1.5395 - sparse_categorical_accuracy: 0.7826 - val_loss: 0.9882 - val_sparse_categorical_accuracy: 0.8342
Epoch 2/200
207/207 [==============================] - ETA: 0s - loss: 0.3577 - sparse_categorical_accuracy: 0.9242
Epoch 2: val_sparse_categorical_accuracy improved from 0.83424 to 0.84239, saving model to ensemble_3.hdf5
207/207 [==============================] - 69s 334ms/step - loss: 0.3577 - sparse_categorical_accuracy: 0.9242 - val_loss: 1.0492 - val_sparse_categorical_accuracy: 0.8424
Epoch 3/200
207/207 [==============================] - ETA: 0s - loss: 0.2417 - sparse_categorical_accuracy: 0.9438
Epoch 3: val_sparse_categorical_accuracy improved from 0.84239 to 0.85870, saving model to ensemb

### 앙상블 V3 fine tune

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)
en_out = Dense(units=37, activation='softmax')(z)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

en_model = create_model3()

moat3.trainable=False
moat4.trainable=False

en_model.load_weights('ensemble_3.hdf5')

moat3.trainable=True
moat4.trainable=True

opt3 = AdaBeliefOptimizer(learning_rate=1e-6, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=15, restore_best_weights=True)])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-25-b59c69fc53d1>:37: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/30
207/207 [==============================] - 322s 849ms/step - loss: 0.1462 - sparse_categorical_accuracy: 0.9671 - val_loss: 0.8472 - val_sparse_categorical_accuracy: 0.8886
Epoch 2/30
207/207 [==============================] - 172s 828ms/step - loss: 0.1451 - sparse_categorical_accuracy: 0.9668 - val_loss: 0.8278 - val_sparse_categorical_accuracy: 0.8886
Epoch 3/30
207/207 [==============================] - 172s 829ms/step - loss: 0.1323 - sparse_categorical_accuracy: 0.9707 - val_loss: 0.8010 - val_sparse_categorical_accuracy: 0.8859
Epoch 4/30
207/207 [==============================] - 171s 824ms/step - loss: 0.1597 - sparse_categorical_accuracy: 0.9656 - val_loss: 0.7832 - val_sparse_categorical_accuracy: 0.8859
Epoch 5/30
207/207 [==============================] - 170s 822ms/step - loss: 0.1639 - sparse_categorical_accuracy: 0.9671 - val_loss: 0.7751 - val_sparse_categorical_accuracy: 0.8940
Epoch 6/30
207/207 [==============================] - 170s 823ms/step - loss: 0.

KeyboardInterrupt: ignored

## 앙상블 V4 : 512layer + (dropout)

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('pet_moat1_final_disjoint1.h5')
moat4.load_weights('pet_moat2_final_disjoint2.h5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)
l= Dense(units=512, activation='gelu')(z)
l= Dropout(0.5)(l)
l= Dense(units=512, activation='gelu')(l)
l= Dropout(0.5)(l)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

opt3 = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05, epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_4.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('ensemble_4.hdf5.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer, keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=20)])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-21-b4f4797eec4c>:45: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/200
207/207 [==============================] - ETA: 0s - loss: 4.0425 - sparse_categorical_accuracy: 0.0900
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.23370, saving model to ensemble_4.hdf5
207/207 [==============================] - 156s 381ms/step - loss: 4.0425 - sparse_categorical_accuracy: 0.0900 - val_loss: 2.7954 - val_sparse_categorical_accuracy: 0.2337
Epoch 2/200
207/207 [==============================] - ETA: 0s - loss: 2.1110 - sparse_categorical_accuracy: 0.4496
Epoch 2: val_sparse_categorical_accuracy improved from 0.23370 to 0.59783, saving model to ensemble_4.hdf5
207/207 [==============================] - 76s 368ms/step - loss: 2.1110 - sparse_categorical_accuracy: 0.4496 - val_loss: 1.5436 - val_sparse_categorical_accuracy: 0.5978
Epoch 3/200
207/207 [==============================] - ETA: 0s - loss: 1.1202 - sparse_categorical_accuracy: 0.7258
Epoch 3: val_sparse_categorical_accuracy improved from 0.59783 to 0.76359, saving model to ensem

KeyboardInterrupt: ignored

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=512, activation='gelu')(z)
l= Dropout(0.5)(l)
l= Dense(units=512, activation='gelu')(l)
l= Dropout(0.5)(l)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

en_model = create_model3()

moat3.trainable=False
moat4.trainable=False

en_model.load_weights('ensemble_4.hdf5')

moat3.trainable=True
moat4.trainable=True

opt3 = AdaBeliefOptimizer(learning_rate=1e-6, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model.compile(optimizer=opt3,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=50,                                             
                    verbose=1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10)])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



<ipython-input-14-b1dd6b04b63a>:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = en_model.fit_generator(train_generator,


Epoch 1/50
207/207 [==============================] - 318s 856ms/step - loss: 0.7762 - sparse_categorical_accuracy: 0.8089 - val_loss: 0.3941 - val_sparse_categorical_accuracy: 0.8967
Epoch 2/50
207/207 [==============================] - 176s 849ms/step - loss: 0.7482 - sparse_categorical_accuracy: 0.8119 - val_loss: 0.3982 - val_sparse_categorical_accuracy: 0.9049
Epoch 3/50
207/207 [==============================] - 176s 849ms/step - loss: 0.6710 - sparse_categorical_accuracy: 0.8345 - val_loss: 0.3909 - val_sparse_categorical_accuracy: 0.9049
Epoch 4/50
207/207 [==============================] - 176s 848ms/step - loss: 0.6907 - sparse_categorical_accuracy: 0.8282 - val_loss: 0.3875 - val_sparse_categorical_accuracy: 0.8995
Epoch 5/50
207/207 [==============================] - 175s 847ms/step - loss: 0.6531 - sparse_categorical_accuracy: 0.8412 - val_loss: 0.3912 - val_sparse_categorical_accuracy: 0.9022
Epoch 6/50
207/207 [==============================] - 176s 848ms/step - loss: 0.

## 모델 평가 코드

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report

batch_size = 50
num_of_test_samples = nb_test_samples

with tf.device('/device:GPU:0'):
  predictions = en_model.predict_generator(test_generator)

y_pred = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

print(class_labels)
print(confusion_matrix(test_generator.classes, y_pred))
report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

<ipython-input-52-5cb7185c8b4e>:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = en_model.predict_generator(test_generator)


['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '4', '5', '6', '7', '8', '9']
[[87  1  0 ...  0  1  0]
 [ 1 82  0 ...  0  1  0]
 [ 0  0 85 ...  0  0  0]
 ...
 [ 0  0  0 ... 74  0  0]
 [ 0  0  0 ...  0 78  1]
 [ 0  0  0 ...  0  0 82]]
              precision    recall  f1-score   support

           1       0.88      0.89      0.88        98
          10       0.88      0.82      0.85       100
          11       0.93      0.85      0.89       100
          12       0.88      0.90      0.89        97
          13       0.94      0.93      0.93       100
          14       0.95      0.92      0.93       100
          15       0.91      0.98      0.94       100
          16       1.00      0.96      0.98       100
          17       0.90      0.93      0.92       100
          18       0.97      1.00      0.99       100
          19       0.97      0.95  